<a href="https://colab.research.google.com/github/SepideSohrabi/K-fold-Image-Classification/blob/main/2th_customized_k_fold_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
#C:\Users\sadma\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\client\session.py:1766: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  #warnings.warn('An interactive session is already active. This can '

In [2]:


# Make it False if do not intend to use Google Colab and want to train in local machine!!
google_colab_flag = True

# For training in Google Colab
if(google_colab_flag):
    from google.colab import drive
    drive.mount('/content/drive')
    !ls
    import sys
    # This is the path to where in google drive the code is stored!
    
    root_path = '/content/drive/MyDrive/AmnAzmoonPics/'
    sys.path.append(root_path)

# For local training
else:
    root_path = ''



Mounted at /content/drive
drive  sample_data


In [3]:
 root_path

'/content/drive/MyDrive/AmnAzmoonPics/'

In [12]:
from PIL import Image
import glob
image_legal = []
for filename in glob.glob('/content/drive/MyDrive/AmnAzmoonPics/DS/train/Legal/*.png'): #assuming gif
    im=Image.open(filename)
    image_legal.append(im)

In [13]:
image_ilegal = []
for filename in glob.glob('/content/drive/MyDrive/AmnAzmoonPics/DS/train/iLegal/*.png'): #assuming gif
    im=Image.open(filename)
    image_ilegal.append(im)

In [ ]:
#for preprocessing 
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])


In [ ]:
X_iLegal=[]
for img in image_ilegal:
    transformed_img=transform(img)
    X_iLegal.append(transformed_img)


In [ ]:
X_Legal=[]
for img in image_legal:
  transformed_img=transform(img)
  X_Legal.append(transformed_img)

In [ ]:
import torch 
# Load the ResNext-WSL model

# small size model
#model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x8d_wsl')
# medium size
model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x16d_wsl')
# large size
# model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x32d_wsl')
# very large
# model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x48d_wsl')
# model.eval()

In [ ]:
model.eval()
out_Legal=[]
for i in range (len(X_Legal)):
    batch_t = torch.unsqueeze(X_Legal[i], 0)
    out_Legal.append(model(batch_t).detach().numpy())


In [ ]:
out_iLegal=[]
for i in range (len(X_iLegal)):
    batch_t = torch.unsqueeze(X_iLegal[i], 0)
    out_iLegal.append(model(batch_t).detach().numpy())


In [ ]:
out_Legal_array=np.asarray(out_Legal)
out_iLegal_array=np.asarray(out_iLegal)

In [ ]:
out_Legal_array=out_Legal_array[..., np.newaxis]
out_iLegal_array=out_iLegal_array[..., np.newaxis]

In [ ]:
#1 refers to legal
#0 refers to ilegal

for i in range (45):
  out_Legal_array[i , : , : , 0] = 1


for i in range (46):
  out_iLegal_array[i , : , : , 0] = 0

In [ ]:
My_Dataset=np.concatenate(( out_Legal_array ,out_iLegal_array ),axis=0)

In [ ]:
def prepare_name_with_labels(folder_name, dataset_type='train'):
    """ Prepare the file names (X) and the class labels (Y) from folder location of images.

        Args:
            folder_name: str
                Source folder's path

    """
    source_files = os.listdir(os.path.join(dataset_folder_name, dataset_type, folder_name))
    y_label = 0
    for i in range(len(class_labels)):
        if(folder_name == class_labels[i]):
            y_label = i
    for val in source_files:
        X.append(val)
        Y.append(y_label)

In [ ]:
# Organize file names and class labels in X and Y variables
for i in range(len(class_labels)):
    prepare_name_with_labels(class_labels[i])

In [ ]:


X = np.asarray(X)
Y = np.asarray(Y)



In [ ]:
batch_size = 512
epoch = 100 #اینجا رو از صد گذاشتم ده که زمان نگیره 
num_of_channels = 3
number_of_class_labels = len(class_labels)

In [ ]:
"""
    Note that, this model structure is a very basic one. 
    To achieve better performance, you should change the model structure and hyperparameters according to your needs and data. 
    So, optimize the structure of the model!
"""

def get_model():
    activation_function = 'relu'

    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same',
                     activation=activation_function, input_shape=(img_rows, img_cols, num_of_channels)))
    model.add(Conv2D(64, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(32, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(16, (3, 3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(16, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(number_of_class_labels, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


model = get_model()

In [ ]:
#Stratified K-Fold Cross validation

In [ ]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X, Y)
fold_num = 0
for train_index, val_index in skf.split(X, Y):
    # First cut all images from validation to train (if any exists)
    transfer_all_class_between_folders('validation', 'train', 1.0)
    fold_num += 1
    print("Results for fold", fold_num)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for each_index in range(len(X_val)):
        class_label = ''
        for i in range(len(class_labels)):
            if(Y_val[each_index] == i):
                class_label = class_labels[i]
        # Then, copy the validation images to the validation folder
        shutil.move(os.path.join(dataset_folder_name, 'train', class_label, X_val[each_index]),
                    os.path.join(dataset_folder_name, 'validation', class_label, X_val[each_index]))

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.20,
        fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)
    
    # fit model
    history = model.fit(train_generator,
                        epochs=epoch)

    predictions = model.predict(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    
    # evaluate validation performance
    print("***Performance on Validation data***")
    val_acc, val_prec, val_fScore = my_metrics(true_classes, y_predictions)

Results for fold 1
Found 62 images belonging to 2 classes.
Found 32 images belonging to 3 classes.
Epoch 1/100
1/1 [==============================] - 10s 10s/step - loss: 0.6921 - accuracy: 0.4839
Epoch 2/100
1/1 [==============================] - 8s 8s/step - loss: 0.6985 - accuracy: 0.5484
Epoch 3/100
1/1 [==============================] - 9s 9s/step - loss: 0.6937 - accuracy: 0.5484
Epoch 4/100
1/1 [==============================] - 8s 8s/step - loss: 0.6864 - accuracy: 0.5323
Epoch 5/100
1/1 [==============================] - 8s 8s/step - loss: 0.7017 - accuracy: 0.5161
Epoch 6/100
1/1 [==============================] - 8s 8s/step - loss: 0.7056 - accuracy: 0.4839
Epoch 7/100
1/1 [==============================] - 11s 11s/step - loss: 0.6956 - accuracy: 0.5323
Epoch 8/100
1/1 [==============================] - 8s 8s/step - loss: 0.6864 - accuracy: 0.5000
Epoch 9/100
1/1 [==============================] - 8s 8s/step - loss: 0.7057 - accuracy: 0.4516
Epoch 10/100
1/1 [===============

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Found 31 images belonging to 3 classes.
Epoch 1/100
1/1 [==============================] - 8s 8s/step - loss: 0.5616 - accuracy: 0.7619
Epoch 2/100
1/1 [==============================] - 8s 8s/step - loss: 0.4981 - accuracy: 0.8095
Epoch 3/100
1/1 [==============================] - 8s 8s/step - loss: 0.6409 - accuracy: 0.7619
Epoch 4/100
1/1 [==============================] - 8s 8s/step - loss: 0.5410 - accuracy: 0.7937
Epoch 5/100
1/1 [==============================] - 8s 8s/step - loss: 0.5188 - accuracy: 0.7619
Epoch 6/100
1/1 [==============================] - 8s 8s/step - loss: 0.5974 - accuracy: 0.7143
Epoch 7/100
1/1 [==============================] - 8s 8s/step - loss: 0.4689 - accuracy: 0.7937
Epoch 8/100
1/1 [==============================] - 10s 10s/step - loss: 0.4710 - accuracy: 0.8095
Epoch 9/100
1/1 [==============================] - 8s 8s/step - loss: 0.4918 - accuracy: 0.7619
Epoch 10/100
1/1 [==============================] - 8s 8s/step - loss: 0.4937 - accuracy: 0.73

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/100
1/1 [==============================] - 9s 9s/step - loss: 0.4515 - accuracy: 0.8254
Epoch 2/100
1/1 [==============================] - 8s 8s/step - loss: 0.3354 - accuracy: 0.8413
Epoch 3/100
1/1 [==============================] - 8s 8s/step - loss: 0.3613 - accuracy: 0.8095
Epoch 4/100
1/1 [==============================] - 8s 8s/step - loss: 0.3259 - accuracy: 0.8413
Epoch 5/100
1/1 [==============================] - 8s 8s/step - loss: 0.3343 - accuracy: 0.8254
Epoch 6/100
1/1 [==============================] - 8s 8s/step - loss: 0.3979 - accuracy: 0.7460
Epoch 7/100
1/1 [==============================] - 8s 8s/step - loss: 0.2864 - accuracy: 0.8254
Epoch 8/100
1/1 [==============================] - 8s 8s/step - loss: 0.2921 - accuracy: 0.8730
Epoch 9/100
1/1 [==============================] - 8s 8s/step - loss: 0.3631 - accuracy: 0.8571
Epoch 10/100
1/1 [==============================] - 8s 8s/step - loss: 0.3650 - accuracy: 0.8254
Epoch 11/100
1/1 [=====================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

# save the model to disk
import pickle
filename = 'finalized_model.h5'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

SyntaxError: ignored

In [ ]:
import cv2

In [ ]:
img = (cv2.imread("/content/8.png"))/255.0

In [ ]:
img = np.resize(img ,(1,100,100,3))

In [ ]:
model.predict(img)

1/1 [==============================] - 0s 40ms/step


array([[0.19827849, 0.8017215 ]], dtype=float32)